# Определение вероятности наличия сердечно-сосудистых заболеваний по данным первичного осмотра

Используем датасет с данными по сердечно-сосудистым заболеваниям, доступный на Kaggle https://www.kaggle.com/raminhashimzade/cardio-disease

В датасете 12 столбцов, из которых 11 признаков и последний столбец с целевой переменной - нужно определить вероятность наличия сердечно-сосудистых заболеваний у пациентов. В датасете 70 тысяч строк.

Обучим модель и создадим rest api сервис, к которому можно будет обращаться для получения прогнозов.

## Подключение библиотек и скриптов

In [2]:
#! pip install dill

In [14]:
%matplotlib inline

import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import dill

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from catboost import CatBoostClassifier

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

## Чтение данных

In [2]:
df = pd.read_csv('data/train_case2.csv', ';', index_col='id')
df.head(3)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [4]:
print(df.shape)

(70000, 12)


### Описание датасета

* `age` - возраст в днях
* `gender` - пол (1 - женский, 2 - мужской)
* `height` - рост, см
* `weight` - вес, кг
* `ap_hi` - систолическое кровяное давление
* `ap_lo` - дистолическое кровяное давление
* `cholesterol` - холестерол (1: норма, 2: выше нормы, 3: значительно выше нормы)
* `gluc` - глюкоза (1: норма, 2: выше нормы, 3: значительно выше нормы)
* `smoke` - курит или нет пациент (0 = нет, 1 = да))
* `alco` - алкоголь (0 = нет, 1 = да)
* `active` - активность (0 = пассивная жизнь, 1 = активная жизнь)
* `cardio` - целевая переменная (0 = нет, 1 = да)

### Разбивка  на Train и Test

Разделим данные на train/test и сохраним тестовую выборку на диск.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("data/X_test.csv", index=None)
y_test.to_csv("data/y_test.csv", index=None)
#save train
X_train.to_csv("data/X_train.csv", index=None)
y_train.to_csv("data/y_train.csv", index=None)

### Предподготовка данных

К полям:
- `gender`, `cholesterol` применим OHE-кодирование
- `age`, `height`, `weight`, `ap_hi`, `ap_lo` применим StandardScaler
- `gluc`, `smoke`, `alco`, `active` - оставим как есть

In [18]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [19]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

In [20]:
continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [21]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

## Модель

In [22]:
%%time

frozen_params = {
    'eval_metric': 'F1',
    'auto_class_weights': 'Balanced',
    'silent': True,
    'one_hot_max_size': 20,
    'early_stopping_rounds': 50,
    'boosting_type': 'Ordered',
    'allow_writing_files': False
}

pipeline = Pipeline([
    ('features',feats),
    ('classifier', CatBoostClassifier(
    **frozen_params,
    depth=4,
    iterations=400,
    learning_rate=0.1,
    l2_leaf_reg=2.5,
    bagging_temperature=1.5
)),
])

Wall time: 5 ms


## Обучение модели

Обучим пайплайн на всем тренировочном датасете

In [23]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

Сохраним модель (пайплайн)

In [24]:
with open("catboost_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

## Проверка работоспособности и качества пайплайна
Здесь не запускаем API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [25]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [26]:
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv")

In [27]:
X_test.head(3)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,21770,1,156,64.0,140,80,2,1,0,0,1
1,21876,1,170,85.0,160,90,1,1,0,0,1
2,23270,1,151,90.0,130,80,1,1,0,0,1


In [28]:
with open('catboost_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [29]:
predictions = pipeline.predict_proba(X_test.iloc[:500])
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_prediction.csv", index=None)

In [30]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:500])

0.8262755265347929